In [2]:
# libraries
import os
import sys
import pandas as pd
import numpy as np
import datetime
from matplotlib import pyplot as plt
import seaborn as sns
from IPython.display import display, Markdown

plt.rcParams.update({'figure.max_open_warning': 0})

In [ ]:
def fast_plot(data, rolling=1, alpha=0.5, figsize=(12, 4), new_figure=True, title=None, label=None):
    data = data.rolling(rolling).mean()
    with plt.style.context(base_context):
        if new_figure:
            plt.figure(figsize=figsize)
        plt.scatter(data.index, data, alpha=alpha)
        plt.plot(data.index, data, linewidth=2, label=label)
        if title is not None:
            plt.title(title)


def fast_legend(ncol=3, pos=(0.5, -0.15), fontsize=16):
    return plt.legend(loc='upper center', ncol=ncol, bbox_to_anchor=pos, fontsize=fontsize)

In [3]:
# Считывание данных по 1 кейсу

train_unlabled = pd.read_excel('https://github.com/romanpuzevich/project-mipt-ml/blob/master/project-mipt-ml/data/raw/first_case/train_unlabeled.xlsx', index_col='Параметр', parse_dates=True, engine='openpyxl')
train_labeled = pd.read_excel('https://github.com/romanpuzevich/project-mipt-ml/blob/master/project-mipt-ml/data/raw/first_case/train_labeled.xlsx', index_col='Параметр', parse_dates=True, engine='openpyxl')

BadZipFile: ignored

In [ ]:
# Работа с неразмеченным датасетом
# Просмотр колонок
print(train_unlabeled.columns)
# Переименование для удобства
train_unlabeled.rename(columns = {'Параметр':'Дата_время'}, inplace = True)
# Выявлено: отсутствие колонок x003, x004, x027, x028

# Типы колонок и null значения
display(train_unlabeled.info())
# Выявлено: 
# 1. Дата_время datetime64[ns], остальные float64
# 2. Нет np.NaN значений в данных

# Общая статистика
display(train_unlabeled.describe().T)
# Выявлено:
# 1. В x008, x010, x023, x024, x025, x029, x048 подозрения на выбросы в данных - отрицательное минимальное значение
# 2. В x002, x017, x036, x038, x039 подозрения на выбросы - очень большие значения max 

# Работа с размеченным датасетом и сравнение
display(train_labeled.describe().T)
# Выявлены следующие расхождения:
# 1. В столбцах представленных выше действительно присутсвуют выбросы, подозрения подтвердились
# 2. Стоит так же обратить внимание на все столбцы, где минимальное значение меньше 530 или максимальное больше 621

In [ ]:
def get_exp(df, time='Параметр', measurement=pd.to_timedelta('0 days 00:10:00')):
    experiments = {}
    count = 1
    index = sorted(df.index.tolist())
    for i, ind in enumerate(index):
        if i == 0:
            begin = df[f"{time}"][ind]
            continue

        if i == len(df) - 2:
            experiments[count] = [begin, df[f"{time}"][index[i + 1]]]
            break

        if (df[f"{time}"][index[i + 1]] - df[f"{time}"][ind]) > measurement:
            experiments[count] = [begin, df[f"{time}"][ind]]
            begin = df[f"{time}"][index[i + 1]]
            count += 1

    return experiments


def experimental_stat(unlabeled, labeled):
    columns = ['Experiment', 'Start', 'Duration', 'Total_rows', 'Clear_rows', 'Percantage_release']
    df = pd.DataFrame(columns=columns)

    info = get_exp(unlabeled)
    for i in info.keys():

        rows = unlabeled.query(f"Параметр >= '{info[i][0]}' and Параметр <= '{info[i][1]}'").shape[0]
        clear_rows = labeled.query(f"Параметр >= '{info[i][0]}' and Параметр <= '{info[i][1]}'").shape[0]
        percantage = np.round((1 - clear_rows / rows) * 100, 2)

        values = np.array([i, info[i][0], info[i][1] - info[i][0], rows, clear_rows, percantage]).reshape(1, -1)

        exp = pd.DataFrame(columns=columns, data=values)

        df = pd.concat([df, exp])

    return df

display(experimental_stat(train_unlabeled, train_labeled))
# после разбиения на эксперименты можно заметить,
# что процент выбросов больше в коротких экспериментах -> 
# возможно их больше в начале эксперимента
# нужно построить график

In [ ]:
plt.figure(figsize=(30, 30))
display(Markdown('__Данные без аномалий:__'))
sns.heatmap(train_unlabeled.corr(), annot = True, fmt='.2g');

# можно заметить, что много признаков скореллированы
# это нужно учитывать при выборе метода